# `LinearRegionSelector` with single lines

In [ ]:
import fastplotlib as fpl
import numpy as np
# from ipywidgets import IntRangeSlider, FloatRangeSlider, VBox

fig = fpl.Figure((2, 2))

# preallocated size for zoomed data
zoomed_prealloc = 1_000

# data to plot
xs = np.linspace(0, 10* np.pi, 1_000)
sine = np.sin(xs)
sine += 100

# make sine along x axis
sine_graphic_x = fig[0, 0].add_line(np.column_stack([xs, sine]), offset=(10, 0, 0))

# just something that looks different for line along y-axis
sine_y = sine
sine_y[sine_y > 0] = 0

# sine along y axis
sine_graphic_y = fig[0, 1].add_line(np.column_stack([sine_y, xs]))

# offset the position of the graphic to demonstrate `get_selected_data()` later
sine_graphic_y.position_x = 50
sine_graphic_y.position_y = 50

# add linear selectors
ls_x = sine_graphic_x.add_linear_region_selector()  # default axis is "x"
ls_y = sine_graphic_y.add_linear_region_selector(axis="y")

# preallocate array for storing zoomed in data
zoomed_init = np.column_stack([np.arange(zoomed_prealloc), np.zeros(zoomed_prealloc)])

# make line graphics for displaying zoomed data
zoomed_x = fig[1, 0].add_line(zoomed_init)
zoomed_y = fig[1, 1].add_line(zoomed_init)


def interpolate(subdata: np.ndarray, axis: int):
    """1D interpolation to display within the preallocated data array"""
    x = np.arange(0, zoomed_prealloc)
    xp = np.linspace(0, zoomed_prealloc, subdata.shape[0])
    
    # interpolate to preallocated size
    return np.interp(x, xp, fp=subdata[:, axis])  # use the y-values

@ls_x.add_event_handler("selection")
def set_zoom_x(ev):
    """sets zoomed x selector data"""
    # get the selected data
    selected_data = ev.get_selected_data()
    if selected_data.size == 0:
        # no data selected
        zoomed_x.data[:, 1] = 0

    # set the y-values
    zoomed_x.data[:, 1] = interpolate(selected_data, axis=1)
    fig[1, 0].auto_scale()


def set_zoom_y(ev):
    """sets zoomed x selector data"""
    # get the selected data
    selected_data = ev.get_selected_data()
    if selected_data.size == 0:
        # no data selected
        zoomed_y.data[:, 0] = 0

    # set the x-values
    zoomed_y.data[:, 0] = -interpolate(selected_data, axis=1)
    fig[1, 1].auto_scale()


fig.show(maintain_aspect=False)

In [ ]:
sine_graphic_x.data[:, 1].ptp()

In [ ]:
np.clip(-0.1, 0, 10)

In [ ]:
ls_y.selection

In [ ]:
ls_y.get_selected_indices()

In [ ]:
# data to plot
xs = np.linspace(0, 100, 1_000)
sine = np.sin(xs) * 20
cosine = np.cos(xs) * 20

fig_stack = fpl.Figure((5, 1))

# sines and cosines
sines = [sine] * 2
cosines = [cosine] * 2

# make line stack
line_stack = fig_stack[0, 0].add_line_stack(sines + cosines, separation=50)

# make selector
selector = line_stack.add_linear_region_selector()

# populate subplots with preallocated graphics
for i, subplot in enumerate(fig_stack):
    if i == 0:
        # skip the first one
        continue
    # make line graphics for displaying zoomed data
    subplot.add_line(zoomed_init, name="zoomed")


@selector.add_event_handler("selection")
def update_zoomed_subplots(ev):
    """update the zoomed subplots"""
    zoomed_data = ev.get_selected_data()
    
    for i in range(len(zoomed_data)):
        # interpolate y-vals
        data = interpolate(zoomed_data[i], axis=1)
        fig_stack[i + 1, 0]["zoomed"].data[:, 1] = data
        fig_stack[i + 1, 0].auto_scale()


fig_stack.show()

# Large line stack with selector